# Title

## Setup

In [12]:
# --- Configture Notebook ------
# show all outputs of cell
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=100,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

# enable automatic reloading
%load_ext autoreload
%autoreload 2

from pathlib import Path

from arxiv_article_classifier.utils import display_fully
from arxiv_article_classifier.data.scrape_arxiv import CATEGORIES_OF_INTEREST

from arxiv_article_classifier.data.make_interim_data import make_interim_data
from arxiv_article_classifier.data.load import load_processed_data, load_taxonomy

from pandas.core.base import PandasObject

PandasObject.display_fully = display_fully

DATAFOLDER = Path().cwd().parent / "data"

import pandas as pd
import ast
import pickle

from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# load taxonomy
taxonomy = load_taxonomy(DATAFOLDER / "raw" / "taxonomy.pkl")

In [3]:
df = pd.read_csv(DATAFOLDER / "raw" / "articles.csv").assign(
    tags=lambda df: df["tags"].apply(lambda x: ast.literal_eval(x))
)
df.head()

,ids,titles,abstracts,tags
0,http://arxiv.org/abs/2210.00982v1,Assuring safety of vision-based swarm formatio...,Vision-based formation control systems recentl...,"[cs.MA, cs.RO, cs.SE]"
1,http://arxiv.org/abs/2210.01163v2,Agent swarms: cooperation and coordination und...,Here we consider the communications tactics ap...,[cs.MA]
2,http://arxiv.org/abs/2210.01314v1,Meta Navigation Functions: Adaptive Associatio...,"In this paper, we introduce a new class of pot...","[cs.RO, cs.MA]"
3,http://arxiv.org/abs/2210.01452v1,Federated Reinforcement Learning for Real-Time...,With the recent advances in mobile energy stor...,"[eess.SY, cs.LG, cs.MA, cs.SY]"
4,http://arxiv.org/abs/2210.01662v1,DGORL: Distributed Graph Optimization based Re...,An optimization problem is at the heart of man...,"[cs.RO, cs.MA]"


In [4]:
unique_tags = {cat for catlist in df["tags"] for cat in catlist}
len(unique_tags)

list(unique_tags)[:10]

1399

['65M12, 74H15, 74H80, 74B20, 74H55, 74H30',
 '65B99, 65F10, 65F20, 65F45, 65F55, 68W20, 68W25, 94A20, 68P30, 68P20',
 '62M20, 62F12, 62E20',
 '49K15, 49M99, 90C30, 93B40, 93C55',
 'F.2',
 'nlin.CD',
 '49J45, 35B44, 47J06, 35A35',
 '65C05, 90C15, 35R60, 90C48, 90C30, 60H25, 49M41, 35Q93',
 'D.2.5; K.3.2',
 '37N30, 65N21, 65L09, 86A22']

In [5]:
with open(DATAFOLDER / "raw" / "taxonomy.pkl", "rb") as f:
    taxonomy = pickle.load(f)
categories = taxonomy.keys()

df["tags"] = df["tags"].map(lambda tags: [tag for tag in tags if tag in categories])

print(
    f"{len({cat for catlist in df['tags'] for cat in catlist})} found out of {len(categories)} existing categories."
)

mlb = MultiLabelBinarizer()

message_tags_matrix = pd.DataFrame(mlb.fit_transform(df["tags"]), columns=mlb.classes_)

fig = px.bar(
    pd.DataFrame(
        pd.DataFrame(message_tags_matrix, columns=mlb.classes_).sum()
        / message_tags_matrix.shape[0]
        * 100,
        columns=["perc_articles"],
    )
    .sort_values(by="perc_articles", ascending=False)
    .assign(is_category_of_interest=lambda df: df.index.map(lambda x: x in CATEGORIES_OF_INTEREST)),
    title="Which percentage of abstracts has which tag?",
    color="is_category_of_interest",
)
_ = fig.update_xaxes(tickangle=45)
fig.show()

fig = px.bar(
    pd.DataFrame(message_tags_matrix.sum(axis=1)).groupby(0).size()
    / message_tags_matrix.shape[0]
    * 100,
    title="Percentage of articles with n tags",
)
fig.show()

143 found out of 155 existing categories.


In the first figure, we notice that there are a few tags that have a lot of associated articles even though these tags were not used when creating the dataset. These are in particular cs.SY (systems and control), cs.NA (numerical analysis) and stat.TH (statistics theory) which are aliases for eees.SY, math.NA and math.ST respectively and to some extend cs.SD (sound), which should overlap with audio and speech processing and stat.ML (machine learning). We also ignore cs.AI (artificial intelligence), which seems like not such a useful distinction for our purposes.

In [16]:
df_sum = (
    pd.DataFrame(
        pd.DataFrame(message_tags_matrix, columns=mlb.classes_).sum(),
        columns=["n_articles"],
    )
    .sort_values(by="n_articles", ascending=False)
    .cumsum()
    .assign(perc_tags=lambda df: df["n_articles"] / df["n_articles"].max())
)
df_sum.head(15)

,n_articles,perc_tags
cs.LG,3317,0.185224
cs.CL,4794,0.267702
cs.CV,6156,0.343757
cs.RO,7366,0.411325
math.OC,8550,0.477440
eess.SY,9694,0.541322
cs.CY,10795,0.602803
eess.SP,11851,0.661771
cs.SI,12871,0.718729
math.NA,13889,0.775575


In [7]:
fig = px.area(df_sum, y="perc_tags", title="Percentage of tags captured")
fig.show()
_ = fig.update_xaxes(tickangle=45)

Let's clean the dataset by reducing the tags to the tags of interest. 

In [8]:
df["tags_filtered"] = df["tags"].map(
    lambda tags: [tag for tag in tags if tag in CATEGORIES_OF_INTEREST]
)

print(
    f"{len({cat for catlist in df['tags_filtered'] for cat in catlist})} found out of {len(categories)} existing categories."
)

mlb = MultiLabelBinarizer()

message_tags_matrix = pd.DataFrame(mlb.fit_transform(df["tags_filtered"]), columns=mlb.classes_)

fig = px.bar(
    pd.DataFrame(
        pd.DataFrame(message_tags_matrix, columns=mlb.classes_).sum()
        / message_tags_matrix.shape[0]
        * 100,
        columns=["perc_articles"],
    ).sort_values(by="perc_articles", ascending=False),
    title="Which percentage of abstracts has which tag?",
)
_ = fig.update_xaxes(tickangle=45)
fig.show()

fig = px.bar(
    pd.DataFrame(message_tags_matrix.sum(axis=1)).groupby(0).size()
    / message_tags_matrix.shape[0]
    * 100,
    title="Percentage of articles with n tags",
)
fig.show()

14 found out of 155 existing categories.


## Split into Train, Dev and Test

This is a multilabel dataset. There are multiple ways of how one can pose a multilabel classification problem:

* convert a 
* 
*

Here, I will convert the multilabel into multiple, single-label classification tasks. For each single classification task, the data is quite imbalanced. Therefore, we need to apply a stratified 

In [9]:
make_interim_data(
    input_file=DATAFOLDER / "raw" / "articles.csv",
    output_folder=DATAFOLDER / "interim",
    categories_to_keep=CATEGORIES_OF_INTEREST,
)

Quick check that the stratified split produced reasonable results:

In [10]:
from collections import Counter


(_, _, _, y_train, y_val, y_test), _ = load_processed_data(Path().cwd().parent / "data" / "interim")

pd.DataFrame(
    {
        "train": Counter(
            str(combination)
            for row in get_combination_wise_output_matrix(y_train, order=2)
            for combination in row
        ),
        "validation": Counter(
            str(combination)
            for row in get_combination_wise_output_matrix(y_val, order=2)
            for combination in row
        ),
        "test": Counter(
            str(combination)
            for row in get_combination_wise_output_matrix(y_test, order=2)
            for combination in row
        ),
    }
).fillna(0).display_fully()

,train,validation,test
"(6, 6)",726.00,242.00,242.00
"(5, 5)",600.00,200.00,200.00
"(5, 6)",124.00,41.00,41.00
"(4, 4)","1,990.00",664.00,663.00
"(4, 5)",208.00,70.00,69.00
"(7, 7)",612.00,204.00,204.00
"(2, 7)",94.00,31.00,31.00
"(5, 7)",21.00,7.00,7.00
"(2, 2)",661.00,220.00,220.00
"(2, 5)",19.00,6.00,6.00


Looks good. Let's go to the next notebook, where we explore the text data.